# Notebook to HTML export

## Utils

In [1]:
# Imports
import os
import itertools as it
import base64
import json
import re

import markdown
from markdown.inlinepatterns import SimpleTextPattern
from markdown.extensions import Extension
from markdown.extensions.toc import TocExtension
from markdown.extensions.footnotes import *
from pygments import highlight
from pygments.lexers import PythonLexer
from pygments.formatters import HtmlFormatter

import requests
import plotly

In [2]:
# Util functions

def save(fn, data):
    if isinstance(data, list):
        data = ''.join(data)
    mode = 'wb' if isinstance(data, bytes) else 'w'
    with open(fn, mode) as f:
        f.write(data)


def parse_outputs(c, html):

    for o in c['outputs']:
        if o['output_type'] == 'error':
            continue

        elif o['output_type'] == 'stream':
            s = '<div class="output"><pre>' + ''.join(o['text']) + '</pre></div>\n'
            html.append(s)
        
        elif o['output_type'] == 'display_data':           
            """
            # plotly figure outputs
            if 'application/vnd.plotly.v1+json' in o['data']:
                fig_json = json.dumps(o['data']['application/vnd.plotly.v1+json'])
                fig_plotly = plotly.io.from_json(fig_json)
                fig_html = plotly.io.to_html(fig_plotly, {'responsive': True, 'displayModeBar': False}, 
                                             include_plotlyjs=False, full_html=False, validate=False)
                html.append(fig_html)
            """
                
            # Ipython HTML display outputs
            if 'text/html' in o['data']:
                for s in o['data']['text/html']:
                    html.append(s)

            elif 'text/markdown'  in o['data']:
                for s in o['data']['text/markdown']:
                    html.append(markdown.markdown(s))
            else:
                raise NotImplementedError
                
        else:
                raise NotImplementedError
                
    return html

In [3]:
#Custom markdown extension classes

class MathEscapeExtension(Extension):
    def extendMarkdown(self, md):
        md.inlinePatterns.register(SimpleTextPattern(r'(\$.+?\$)'), 'math', 175)

class ReferenceInlineProcessor(FootnoteInlineProcessor):
    """ InlinePattern for reference markers in a document's body text. """
    def handleMatch(self, m, data):
        id = m.group(1)
        if id in self.footnotes.footnotes.keys():
            a_ext = etree.Element("a")
            a = etree.SubElement(a_ext, "a")
            a_ext.set('id', self.footnotes.makeFootnoteRefId(id, found=True))
            a.set('href', '#' + self.footnotes.makeFootnoteId(id))
            a.set('class', 'footnote-ref')
            a.text = self.footnotes.getConfig("SUPERSCRIPT_TEXT").format(
                list(self.footnotes.footnotes.keys()).index(id) + 1
            )
            return a_ext, m.start(0), m.end(0)
        else:
            return None, None, None
        
class ReferenceExtension(FootnoteExtension):
    """ Footnote Extension. """
    def extendMarkdown(self, md):
        """ Add pieces to Markdown. """
        md.registerExtension(self)
        self.parser = md.parser
        self.md = md
        # Insert a blockprocessor before ReferencePreprocessor
        md.parser.blockprocessors.register(FootnoteBlockProcessor(self), 'footnote', 17)
        # Insert an inline pattern before ImageReferencePattern
        FOOTNOTE_RE = r'\^([^\^([^]*)\^([^\^([^]*)'  # blah blah [^1] blah
        md.inlinePatterns.register(ReferenceInlineProcessor(FOOTNOTE_RE, self), 'footnote', 175)
        # Insert a tree-processor that would actually add the footnote div
        # This must be before all other treeprocessors (i.e., inline and
        # codehilite) so they can run on the the contents of the div.
        md.treeprocessors.register(FootnoteTreeprocessor(self), 'footnote', 50)
        # Insert a tree-processor that will run after inline is done.
        # In this tree-processor we want to check our duplicate footnote tracker
        # And add additional backrefs to the footnote pointing back to the
        # duplicated references.
        md.treeprocessors.register(FootnotePostTreeprocessor(self), 'footnote-duplicate', 15)
        # Insert a postprocessor after amp_substitute processor
        md.postprocessors.register(FootnotePostprocessor(self), 'footnote', 25)
        
    def makeFootnotesDiv(self, root):
        """ Return div of footnotes as et Element. """
        if not list(self.footnotes.keys()):
            return None
        div = etree.Element("div")
        div.set('class', 'footnote')
        # etree.SubElement(div, "br")
        # etree.SubElement(div, "hr")
        # title = etree.SubElement(div, "h2")
        # title.text = 'References'
        ol = etree.SubElement(div, "ol")
        surrogate_parent = etree.Element("div")
        # Backward compatibility with old '%d' placeholder
        backlink_title = self.getConfig("BACKLINK_TITLE").replace("%d", "{}")
        for index, id in enumerate(self.footnotes.keys(), start=1):
            li = etree.SubElement(ol, "li")
            li.set("id", self.makeFootnoteId(id))
            # Parse footnote with surrogate parent as li cannot be used.
            # List block handlers have special logic to deal with li.
            # When we are done parsing, we will copy everything over to li.
            self.parser.parseChunk(surrogate_parent, self.footnotes[id])
            for el in list(surrogate_parent):
                li.append(el)
                surrogate_parent.remove(el)
            backlink = etree.Element("a")
            backlink.set("href", "#" + self.makeFootnoteRefId(id))
            backlink.set("class", "footnote-backref")
            backlink.set(
                "title",
                backlink_title.format(index)
            )
            backlink.text = FN_BACKLINK_TEXT
            if len(li):
                node = li[-1]
                if node.tag == "p":
                    node.text = node.text + NBSP_PLACEHOLDER
                    node.append(backlink)
                else:
                    p = etree.SubElement(li, "p")
                    p.append(backlink)
        return div

## Website assets and style files

In [4]:
# Create export folder
!mkdir -p docs

In [5]:
# HTML header
def get_html_header(nb_title, nb_names, nb_links, active_nb_name):
    html_prefix = f'''<!DOCTYPE html>
<meta charset="utf-8">
<meta name="viewport" content="width=device-width, initial-scale=1">
<title>{nb_title}</title>
<link rel="stylesheet" href="highlight.css">
<link rel="stylesheet" href="style.css">
<link rel="icon" type="image/svg" href="https://raw.githubusercontent.com/mayalenE/personal-webpage/main/assets/icon/favicon.svg">


<!-- math rendering -->
<script>
MathJax = {{
  tex: {{ inlineMath: [['$', '$']] }}
}};
</script>
<script type="text/javascript" id="MathJax-script" async src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-chtml.js">
</script>
<!-- math rendering -->

<!-- plotly rendering -->
<script src="https://cdnjs.cloudflare.com/ajax/libs/plotly.js/2.18.2/plotly.min.js" integrity="sha512-D52Rvz8mPwpAIIg9bRTFYiyy3GlBIE7kN8wGscKV+EgN8tJB7x7scvLlCBsK2KfYXQvPclyv1uY6E8+0HU+sfA==" crossorigin="anonymous" referrerpolicy="no-referrer"></script>
<!-- plotly rendering -->

<!-- admonition rendering -->
<link rel="stylesheet" href="admonition.css">
<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.2.0/css/all.css">
<!-- admonition rendering -->


<!-- top navigation bar -->
<link rel="stylesheet" href="navbar.css">
<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css">

<script type="text/javascript">
/* Toggle between adding and removing the "responsive" class to topnav when the user clicks on the icon */
function myFunction() {{
  var x = document.getElementById("myTopnav");
  if (x.className === "topnav") {{
    x.className += " responsive";
  }} else {{
    x.className = "topnav";
  }}
}}
</script>

<div class="topnav" id="myTopnav">
    <a href="{active_nb_name}.html" class="logo">
        <img  src="https://raw.githubusercontent.com/mayalenE/personal-webpage/main/assets/icon/favicon.svg" class="logo_img" >
    </a>  '''
    
    for nb_idx, nb_name in enumerate(reversed(nb_names)):
        html_prefix += f'''
    <a href="{nb_links[nb_name]}.html" {'class="active"' if nb_name==active_nb_name else ""}>{nb_name.capitalize()}</a>'''
        if nb_idx < len(nb_names)-1:
            html_prefix += "<span> &#x2022; </span>"  
    
    
    html_prefix += f'''
    <a href="javascript:void(0);" class="icon" onclick="myFunction()">
        <i class="fa fa-bars"></i>
    </a>
</div>
<!--  top navigation bar -->


'''
    
    return html_prefix

In [6]:
%%file docs/style.css

* {
}
html {
    font-size: 16px;
    font-family: 'Lato', sans-serif;
     line-height: 1.6em;
}
body {
  margin: 0 auto;
  max-width: 60rem;
}
div,p,h1,h2,h3, h4 {
    margin: 10px 10px 10px 10px;
}
h1,h2,h3, h4 {
    margin-top: 30px;
    margin-bottom: 10px;
    line-height: 1.3em;
}
h1 {font-size: 3.0rem;}
h2 {font-size: 2.0rem;}
h3 {font-size: 1.5rem;}
h4 {font-size: 1.2rem;}

table {
    width: 100%;
    text-align: left;
    font-size: smaller;
    margin: 10px;
    padding-bottom: 5px;
    padding-top: 5px;
    border-bottom: 1px solid rgba(0, 0, 0, 0.1);
    border-top: 1px solid rgba(0, 0, 0, 0.1);
}

th{
    border-bottom: 1px solid rgba(0, 0, 0, 0.1);
}

table:nth-of-type(1) * {
    padding: 0;
    margin: 0;
}

table:nth-of-type(1) td {
    line-height: 1.6em;
}

table:nth-of-type(1) a {
    text-decoration: none;
    # color: black;
    margin: 0;
}

table:nth-of-type(1) th {
    # font-weight: lighter;
}

table:nth-of-type(1) td:first-child {
    font-weight: bold;
}

.output,.highlight {
    display: block;
    overflow-x: auto;
    margin-top: 10px;
    line-height: 1em;
    padding-bottom: 8px;
}
.code-cell {
    margin: 20px 0 20px 0;
    display: block;
}
.output {
    font-size: 80%;
}
.highlight {
    font-size: 90%;
    line-height: 1.2em;
}
pre {
    display: inline;
}
img,video,iframe {
    display:block;
    margin: auto;
    /* max-width:90%; */
}

.highlight .err {
    border: 0px;
}

#pdemo {
    text-align: center;
    border-bottom: 1px solid rgba(0, 0, 0, 0.1);
}

#repro {
    text-align: left;
}

.colab-root {
    display: inline-block;
    background: rgba(230, 230, 230, 0.75);
    padding: 2px 8px;
    border-radius: 4px;
    font-size: 10px!important;
    text-decoration: none;
    color: #000;
    font-weight: 500;
    border: solid 1px rgba(0, 0, 0, 0.08);
    border-bottom-color: rgba(0, 0, 0, 0.15);
    text-transform: uppercase;
    line-height: 16px;
}

.isDisabled {
  color: currentColor;
  cursor: not-allowed;
  opacity: 0.8;
  color: #aaa;
  text-decoration: none;
  pointer-events: none;
}

span.colab-span {
    background-image: url(github.svg);
    background-repeat: no-repeat;
    background-size: 20px;
    background-position: center right;
    display: inline-block;
    padding-right: 24px;
}

#arxiv {
    background-image: url(arxiv.svg);
    vertical-align: bottom;
    background-repeat: no-repeat;
    background-size: 30px;
    background-position: center;
    padding-left: 40px;
    /* padding-top: 6px; */
    padding-bottom: 20px;
    filter: opacity(0.3);
    cursor: not-allowed;
    /* margin-left: 10px; */
    /*    background-color: #b31b1b;*/
}

#demo {
    font-size: 25px!important;
    font-weight: bold;
    padding: 12px 12px;
    background-color: #4a9cda;
    color: #f5d36b;
    border: solid 1px rgba(0, 0, 0, 0.25);
    border-bottom-color: rgba(0, 0, 0, 0.55);
}

.toc a {
    text-decoration: none;
    color: black;
}

.toc a:hover {
    text-decoration: underline;
    color: black;
}

.toc ul {
    list-style-type: none;
    padding-left: 10px;
/*    list-style-position: inside;*/
/*    padding: 0;*/
/*    margin: 0;*/
}

.toc ul ul {
    padding-left: 15px;
    font-size: smaller;
}


@media(max-width: 1440px) {
    .toc {
        display: none;
    }
}

@media(max-width: 800px) {
    h1 {
      font-size: 1.8rem;
    }

    #reprotext {
      display: none;
    }

}

.toc {
    /* The CSS applied to our floating application */
    border-right: 1px solid rgba(0, 0, 0, 0.1);
    padding-right: 20px;
    position: sticky; /*relative;*/
    /*    bottom: 10px;*/
    background-color: white;
    /*    left: 200px;*/
    float: left;
    margin-left: -300px;
    width: 250px;
    /*left: -200px;*/
}

#colablink {
    padding: 2px 4px;
}

.plotly-graph-div {
    margin: 0px auto;
}

Overwriting docs/style.css


In [7]:
lexer, formatter = PythonLexer(), HtmlFormatter()
save('docs/highlight.css', formatter.get_style_defs('.highlight'))

In [8]:
%%file docs/navbar.css

/* Style the top navigation bar */

    .topnav{
        display: inline;
        position: fixed;
        right: 0;
        top: 0; 
        width: 100%;
        background-color: rgb(255, 255, 255);
        margin: 0px;
        /* padding-right: 50px; */
        /* border-bottom-color: #000;
        border-bottom-width: 1px;
        border-bottom-style: solid; */
    }
      
    body{
        margin-top: 100px;
      }
    

    /* Logo */
    .topnav a.logo {
        float: left;
        display: block;
        color: #000;
      }
    
    img.logo_img{
        width: 50px;
        float: left;
        display: inline-block;
      }

    a.logo:hover {
        background-color: #ddd;
      }
    
    
    
    /* Menu */
  .topnav a {
    overflow: hidden;
    float: right;
    display: block;
    color: #000;
    text-align: center;
    padding: 14px 16px;
    text-decoration: none;
    font-size: 1.5em;
  }
    
  .topnav span {
    overflow: hidden;
    float: right;
    display: block;
    color: #000;
    text-align: center;
    padding: 14px 16px;
    text-decoration: none;
    font-size: 1.5em;
  }
  
  
  /* Change the color of links on hover */
  .topnav a:hover {
    background-color: #ddd;
  }

  
  /* Add an active class to highlight the current page */
  .topnav a.active {
    font-weight: bold;
    text-decoration-line: underline;
    text-decoration-color: #04AA6D;
    text-decoration-thickness: 4px;
  }
  
  /* Hide the link that should open and close the topnav on small screens */
  .topnav .icon {
    display: none;
  }

    /* When the screen is less than 600 pixels wide, hide all links, except for the first one ("Home"). Show the link that contains should open and close the topnav (.icon) */
@media screen and (max-width: 600px) {
.topnav a {
    display: none;
}
.topnav span {
    display: none;
}
.topnav a.icon {
  float: right;
  display: block;
}
.topnav a.icon:hover {
  background-color: rgb(255, 255, 255);
  /* color: black; */
}
.topnav{
  padding-right: 0px;
}
}
  
/* The "responsive" class is added to the topnav with JavaScript when the user clicks on the icon. This class makes the topnav look good on small screens (display the links vertically instead of horizontally) */
@media screen and (max-width: 600px) { 
.topnav.responsive a.icon {
    position: absolute;
    right: 0;
    top: 0;
}
    
/* .topnav.responsive{
  padding-top: 40px;
} */
    
.topnav.responsive a {
    float: none;
    display: block;
    text-align: left;
}
}

Overwriting docs/navbar.css


In [9]:
%%file docs/admonition.css

/* Style the admonition notes */
/* Inspired from https://sphinx-book-theme.readthedocs.io/en/latest/reference/kitchen-sink/admonitions.html */

.admonition {
    position: relative;
    margin: 1em 0;
    padding: 0 .6rem .8rem;
    border-radius: .25rem;
    box-shadow: 0 .2rem .5rem #d8d8d8,0 0 .0625rem #d8d8d8;
}

.admonition-title {
    padding: .4rem .6rem .4rem 2rem;
    margin: 0 -.6rem;
    position: relative;
    font-weight: bold;
}

.admonition>p:last-child {
    margin-bottom: 0;
}

.admonition>.admonition-title:before {
    content: "";
    height: 100%;
    left: 0;
    opacity: .1;
    pointer-events: none;
    position: absolute;
    top: 0;
    width: 100%;
}

.admonition>.admonition-title:after {
    content: "";
    font-family: "Font Awesome 6 Free";
    font-weight: 900;
    height: 1rem;
    left: .5rem;
    opacity: 1;
    position: absolute;
    width: 1rem;
}

.admonition.hint,.admonition.seealso { 
    border-left: .2rem solid #28a745;
    background-color: "white"; 
}

.admonition.hint>.admonition-title:before, .admonition.seealso>.admonition-title:before{
    background-color: #28a745;
}

.admonition.hint>.admonition-title:after {
    color: #28a745;
    content: "\f0eb";
}

.admonition.seealso>.admonition-title:after {
    color: #28a745;
    content: "\f064";
}

Overwriting docs/admonition.css


In [10]:
%%file "docs/github.svg"

<svg width="16" height="16" viewBox="0 0 16 16" fill="none" xmlns="http://www.w3.org/2000/svg">
<path fill-rule="evenodd" clip-rule="evenodd" d="M8 0C3.58 0 0 3.58 0 8C0 11.54 2.29 14.53 5.47 15.59C5.87 15.66 6.02 15.42 6.02 15.21C6.02 15.02 6.01 14.39 6.01 13.72C4 14.09 3.48 13.23 3.32 12.78C3.23 12.55 2.84 11.84 2.5 11.65C2.22 11.5 1.82 11.13 2.49 11.12C3.12 11.11 3.57 11.7 3.72 11.94C4.44 13.15 5.59 12.81 6.05 12.6C6.12 12.08 6.33 11.73 6.56 11.53C4.78 11.33 2.92 10.64 2.92 7.58C2.92 6.71 3.23 5.99 3.74 5.43C3.66 5.23 3.38 4.41 3.82 3.31C3.82 3.31 4.49 3.1 6.02 4.13C6.66 3.95 7.34 3.86 8.02 3.86C8.7 3.86 9.38 3.95 10.02 4.13C11.55 3.09 12.22 3.31 12.22 3.31C12.66 4.41 12.38 5.23 12.3 5.43C12.81 5.99 13.12 6.7 13.12 7.58C13.12 10.65 11.25 11.33 9.47 11.53C9.76 11.78 10.01 12.26 10.01 13.01C10.01 14.08 10 14.94 10 15.21C10 15.42 10.15 15.67 10.55 15.59C13.71 14.53 16 11.53 16 8C16 3.58 12.42 0 8 0Z" fill="#1B1F23"/>
</svg>

Overwriting docs/github.svg


In [11]:
%%file "docs/arxiv.svg"

<svg id="primary_logo" data-name="primary logo" xmlns="http://www.w3.org/2000/svg" viewBox="0 0 246.978 111.119"><path d="M427.571,255.154c1.859,0,3.1,1.24,3.985,3.453,1.062-2.213,2.568-3.453,4.694-3.453h14.878a4.062,4.062,0,0,1,4.074,4.074v7.828c0,2.656-1.327,4.074-4.074,4.074-2.656,0-4.074-1.418-4.074-4.074V263.3H436.515a2.411,2.411,0,0,0-2.656,2.745v27.188h10.007c2.658,0,4.074,1.329,4.074,4.074s-1.416,4.074-4.074,4.074h-26.39c-2.659,0-3.986-1.328-3.986-4.074s1.327-4.074,3.986-4.074h8.236V263.3h-7.263c-2.656,0-3.985-1.329-3.985-4.074,0-2.658,1.329-4.074,3.985-4.074Z" transform="translate(-358.165 -222.27)" fill="#7c7469"/><path d="M539.233,255.154c2.656,0,4.074,1.416,4.074,4.074v34.007h10.1c2.746,0,4.074,1.329,4.074,4.074s-1.328,4.074-4.074,4.074H524.8c-2.656,0-4.074-1.328-4.074-4.074s1.418-4.074,4.074-4.074h10.362V263.3h-8.533c-2.744,0-4.073-1.329-4.073-4.074,0-2.658,1.329-4.074,4.073-4.074Zm4.22-17.615a5.859,5.859,0,1,1-5.819-5.819A5.9,5.9,0,0,1,543.453,237.539Z" transform="translate(-358.165 -222.27)" fill="#7c7469"/><path d="M605.143,259.228a4.589,4.589,0,0,1-.267,1.594L590,298.9a3.722,3.722,0,0,1-3.721,2.48h-5.933a3.689,3.689,0,0,1-3.808-2.48l-15.055-38.081a3.23,3.23,0,0,1-.355-1.594,4.084,4.084,0,0,1,4.164-4.074,3.8,3.8,0,0,1,3.718,2.656l14.348,36.134,13.9-36.134a3.8,3.8,0,0,1,3.72-2.656A4.084,4.084,0,0,1,605.143,259.228Z" transform="translate(-358.165 -222.27)" fill="#7c7469"/><path d="M486.149,277.877l-32.741,38.852c-1.286,1.372-2.084,3.777-1.365,5.5a4.705,4.705,0,0,0,4.4,2.914,4.191,4.191,0,0,0,3.16-1.563l40.191-42.714a4.417,4.417,0,0,0,.042-6.042Z" transform="translate(-358.165 -222.27)" fill="#aa142d"/><path d="M486.149,277.877l31.187-38.268c1.492-1.989,2.2-3.03,1.492-4.723a5.142,5.142,0,0,0-4.481-3.161h0a4.024,4.024,0,0,0-3.008,1.108L472.711,274.6a4.769,4.769,0,0,0,.015,6.53L520.512,332.2a3.913,3.913,0,0,0,3.137,1.192,4.394,4.394,0,0,0,4.027-2.818c.719-1.727-.076-3.438-1.4-5.23l-40.124-47.464" transform="translate(-358.165 -222.27)" fill="#7c7469"/><path d="M499.833,274.828,453.169,224.4s-1.713-2.08-3.524-2.124a4.607,4.607,0,0,0-4.338,2.788c-.705,1.692-.2,2.88,1.349,5.1l40.093,48.422" transform="translate(-358.165 -222.27)" fill="#aa142d"/><path d="M390.61,255.154c5.018,0,8.206,3.312,8.206,8.4v37.831H363.308a4.813,4.813,0,0,1-5.143-4.929V283.427a8.256,8.256,0,0,1,7-8.148l25.507-3.572v-8.4H362.306a4.014,4.014,0,0,1-4.141-4.074c0-2.87,2.143-4.074,4.355-4.074Zm.059,38.081V279.942l-24.354,3.4v9.9Z" transform="translate(-358.165 -222.27)" fill="#7c7469"/></svg>

Overwriting docs/arxiv.svg


## Convert Notebooks to HTML

In [12]:
# Iterate through notebooks cells
nb_fps = {'paper': './notebooks/paper.ipynb', 'tuto1': './notebooks/tuto1.ipynb', 'tuto2': './notebooks/tuto2.ipynb'}
nb_links = {'paper': 'index', 'tuto1': 'tuto1', 'tuto2': 'tuto2'}


for nb_name, nb_fp in nb_fps.items():
#     if 'tuto' in nb_name:
#         continue
        
    nb_json = json.load(open(nb_fp, 'r'))
    nb_title = "Curious Exploration of GRN Competencies"
    
    html_header = get_html_header(nb_title, nb_fps.keys(), nb_links, nb_name)

    html = []

    cells_iter = iter(nb_json['cells'])
    for c in cells_iter:
        source = ''.join(c['source'])
        
        if source.startswith('# !export'):
            break  # article body end
            
        if 'tags' in c['metadata'] and 'remove-cell' in c['metadata']['tags']:
            continue
                        
        if c['cell_type'] == 'markdown':
            html.append(source+'\n')
            

        elif c['cell_type'] == 'code':

            # html.append('\n<div class="code-cell">\n')
            if 'tags' in c['metadata'] and 'remove-input' in c['metadata']['tags']:
                pass
            else:
                if source[:20] == 'if nb_mode == "run":':
                    s=source[20:].replace("\n    ","\n")
                else:
                    s=source
                html.append(highlight(s, lexer, formatter)+'\n')
            if 'tags' in c['metadata'] and 'remove-output' in c['metadata']['tags']:
                pass
            else:
                html = parse_outputs(c, html)
            html.append('\n')
            # html.append('</div>\n')
    # print(html[7])
    nb_html = markdown.markdown("".join(html), extensions=['admonition', 'tables', TocExtension(toc_depth='2-4'), MathEscapeExtension(), ReferenceExtension(SUPERSCRIPT_TEXT="[{}]")])

    # adding collapsing sections
    nb_html = nb_html.replace("<!-- end_details -->","</details>")
    nb_html = nb_html.replace("<!-- start_details -->","<details><summary>Read more</summary>")

    # add isDisbaled class to the colab buttons
    # nb_html = nb_html.replace("colab-root","colab-root isDisabled")
    
    
    save("docs/"+nb_links[nb_name]+'.html', html_header+nb_html)